# Model Training
###### by Wilson Lee
###### Data Set : China Mobile User Gemographics Source
###### Link : https://www.kaggle.com/chinapage/china-mobile-user-gemographics

This code will develope a predictor for the the interest vector for the provided database. For a given combination of User age, gender, location, hour of day, and day of week, the model will predict a user interest vector using regression.